In [ ]:
import pandas as pd, numpy as np,xarray as xr
from pathlib import Path
import re, yaml, copy, json
import helper, events_methods

In [ ]:
base = Path("/home/julienb/Documents/database_scripts/database_scripts_test/fiberevent_files/Rat/Rat101_0729_opto_01/")
event_path = helper.singleglob(base, "Events.csv")
info_path = helper.singleglob(base, "*.yaml", search_upward_limit=Path("/home/julienb/Documents/database_scripts/database_scripts_test/fiberevent_files/"))
res_events_path = base/"events.tsv"
info_path

In [ ]:
event_df = pd.read_csv(event_path, sep=",").rename(columns=dict(Name="channel_name"))
event_df.insert(0, "t", event_df.pop("TimeStamp")/1000)
event_df

In [ ]:
info = yaml.safe_load(info_path.open("r"))
info

In [ ]:
event_spec = events_methods.FiberEventProcessing.process_info(event_df["channel_name"].drop_duplicates().to_list(), info["processing"])
pd.DataFrame(list(event_spec.values()))

In [ ]:
all=[]
for ev_name, item in event_spec.items():
    ev_dataframe = events_methods.FiberEventProcessing.compute_evdataframe(event_df, item)
    events = events_methods.FiberEventProcessing.call(item["method"],ev_dataframe, item)
    if len(events.index)!=0:
        all.append(events)
all = pd.concat(all).sort_values("t")[["event_name", "t", "duration", "n_segments","metadata", "waveform_changes", "waveform_values"]]
all

In [ ]:
if "display" in info and "rename" in info["display"]:
    all["event_name"] = all["event_name"].map(lambda e: info["display"]["rename"][e] if e in info["display"]["rename"] else e)
json_cols = ["metadata", "waveform_changes", "waveform_values"]
for col in json_cols:
    all[f"{col}_json"] = all[col].apply(lambda d: json.dumps(d))
all.drop(columns=json_cols).to_csv(res_events_path, sep="\t", index=False)
reloaded = pd.read_csv(res_events_path, sep="\t", index_col=False)
for col in reloaded.columns:
    if col.endswith("_json"):
        reloaded[col[:-4]] = reloaded.pop(col).apply(lambda s: json.loads(s) if not pd.isna(s) else None)
reloaded